In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing as mp

from depsurf import VERSIONS_ARCH, VERSIONS_FLAVOR, VERSIONS_REGULAR, VERSION_LAST, prep_dwarf

with mp.Pool(mp.cpu_count() - 1, maxtasksperchild=1) as pool:
    pool.map(
        prep_dwarf,
        # [VERSION_LAST],
        VERSIONS_REGULAR + VERSIONS_FLAVOR + VERSIONS_ARCH,
    )

[decorator.py:17 ] INFO: dump_funcs       Using /Users/szhong/Downloads/bpf-study/data/funcs/4.8.0-22-generic-amd64.jsonl
[decorator.py:17 ] INFO: dump_funcs       Using /Users/szhong/Downloads/bpf-study/data/funcs/4.18.0-10-generic-amd64.jsonl
[decorator.py:17 ] INFO: dump_funcs       Using /Users/szhong/Downloads/bpf-study/data/funcs/4.4.0-21-generic-amd64.jsonl
[decorator.py:17 ] INFO: dump_funcs       Using /Users/szhong/Downloads/bpf-study/data/funcs/4.13.0-16-generic-amd64.jsonl
[decorator.py:17 ] INFO: dump_funcs       Using /Users/szhong/Downloads/bpf-study/data/funcs/4.10.0-19-generic-amd64.jsonl
[decorator.py:17 ] INFO: dump_funcs       Using /Users/szhong/Downloads/bpf-study/data/funcs/4.15.0-20-generic-amd64.jsonl
[decorator.py:17 ] INFO: dump_funcs       Using /Users/szhong/Downloads/bpf-study/data/funcs/5.0.0-13-generic-amd64.jsonl
[decorator.py:17 ] INFO: dump_funcs       Using /Users/szhong/Downloads/bpf-study/data/funcs/5.15.0-25-generic-amd64.jsonl
[decorator.py:17 ] 